In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pymcdm --quiet

import pandas as pd

# Define base and file paths
base_path = "/content/drive/MyDrive/DAINA 3"
file_path = f"{base_path}/final_score_09_11.csv"


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 5.0 MB/s eta 0:00:00


In [3]:
df = pd.read_csv(file_path)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head(3)


,id,left,top,right,bottom,row_index,col_index,area,perimeter,geometry,land_score,dem_score,dni_score,temp_score,pvout_score,dso_score,pv_score
0,676612,414729.589868,369375.174135,414979.589868,369125.174135,306,855,6656.084138,410.853786,"MULTIPOLYGON (((17.7802286 51.1843321, 17.7801...",1.0,177.928571,1019.047974,9.4,1114.011963,0.034582,0.811841
1,676611,414729.589868,369625.174135,414979.589868,369375.174135,305,855,28107.499155,755.139192,"MULTIPOLYGON (((17.7818396 51.186814799999986,...",1.0,177.714286,1020.970893,9.4,1114.011963,0.033724,0.816066
2,676610,414729.589868,369875.174135,414979.589868,369625.174135,304,855,56502.279292,946.460361,"MULTIPOLYGON (((17.782959 51.188250499999995, ...",1.0,176.648936,1020.913394,9.4,1114.011963,0.032380,0.821032


In [4]:
cols_to_drop = ['left', 'top', 'right', 'bottom', 'row_index', 'col_index', 'geometry']
df_reduced = df.drop(columns=cols_to_drop)
print(df_reduced.shape)
df_reduced.head(3)

(321820, 10)


,id,area,perimeter,land_score,dem_score,dni_score,temp_score,pvout_score,dso_score,pv_score
0,676612,6656.084138,410.853786,1.0,177.928571,1019.047974,9.4,1114.011963,0.034582,0.811841
1,676611,28107.499155,755.139192,1.0,177.714286,1020.970893,9.4,1114.011963,0.033724,0.816066
2,676610,56502.279292,946.460361,1.0,176.648936,1020.913394,9.4,1114.011963,0.032380,0.821032


In [5]:
df_reduced.info()
df_reduced.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321820 entries, 0 to 321819
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           321820 non-null  int64  
 1   area         321820 non-null  float64
 2   perimeter    321820 non-null  float64
 3   land_score   321820 non-null  float64
 4   dem_score    321820 non-null  float64
 5   dni_score    321820 non-null  float64
 6   temp_score   321820 non-null  float64
 7   pvout_score  321820 non-null  float64
 8   dso_score    321820 non-null  float64
 9   pv_score     321820 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 24.6 MB


,0
id,0
area,0
perimeter,0
land_score,0
dem_score,0
dni_score,0
temp_score,0
pvout_score,0
dso_score,0
pv_score,0


In [ ]:
import plotly.express as px

for col in ['area', 'land_score', 'dem_score', 'dni_score','temp_score', 'pvout_score','dso_score','pv_score' ]:
    fig = px.histogram(df_reduced, x=col, nbins=40, title=f"Distribution of {col}", template="plotly_white")
    fig.show()

In [ ]:
import pandas as pd

# Columns you’re analyzing
cols = ['area', 'land_score', 'dem_score', 'dni_score', 'temp_score', 'pvout_score', 'dso_score', 'pv_score']

# Generate descriptive stats
summary = df_reduced[cols].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).T

# Add variance and coefficient of variation (optional)
summary['var'] = df_reduced[cols].var().values
summary['cv (%)'] = (df_reduced[cols].std() / df_reduced[cols].mean() * 100).values

# Rename columns for readability
summary = summary.rename(columns={
    '50%': 'median',
    '25%': 'q1 (25th%)',
    '75%': 'q3 (75th%)',
    '5%': 'p5',
    '95%': 'p95'
})

# Round for clarity
summary = summary[['count', 'mean', 'median', 'min', 'q1 (25th%)', 'q3 (75th%)', 'max', 'std', 'var', 'cv (%)']].round(3)

# Display neatly
summary.style.background_gradient(cmap='Blues').set_caption("Statistical Summary of Key Variables")


,count,mean,median,min,q1 (25th%),q3 (75th%),max,std,var,cv (%)
area,321820.000000,61948.020000,62500.000000,0.010000,62500.000000,62500.000000,62500.000000,5035.059000,25351816.706000,8.128000
land_score,321820.000000,0.242000,0.010000,0.000000,0.000000,0.402000,1.000000,0.352000,0.124000,145.849000
dem_score,321820.000000,250.068000,177.250000,0.000000,133.390000,302.682000,1596.000000,182.949000,33470.423000,73.160000
dni_score,321820.000000,989.237000,999.595000,0.000000,974.160000,1014.117000,1040.597000,52.483000,2754.415000,5.305000
temp_score,321820.000000,8.846000,9.400000,0.000000,8.400000,9.600000,9.900000,1.238000,1.533000,13.996000
pvout_score,321820.000000,1089.825000,1097.942000,0.000000,1084.427000,1111.821000,1134.466000,82.672000,6834.601000,7.586000
dso_score,321820.000000,0.741000,0.763000,0.000000,0.660000,0.850000,1.000000,0.152000,0.023000,20.523000
pv_score,321820.000000,0.248000,0.222000,0.000000,0.130000,0.338000,1.000000,0.156000,0.024000,62.704000


In [ ]:
df_reduced = df_reduced.drop(columns=["area"])

In [1]:
# #Inverting pvscore because DSO (tauron) said that farther away from existing pv farms is better
# df_reduced["pv_score"] = 1 - df_reduced["pv_score"]

In [ ]:
import pandas as pd
import plotly.express as px


corr_cols = ['land_score', 'dem_score', 'dni_score',
             'temp_score', 'pvout_score', 'dso_score', 'pv_score']

corr_matrix = df_reduced[corr_cols].corr()

# Display numeric correlation table
display(corr_matrix.round(3))

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title='Correlation Matrix of MCDM Criteria',
    aspect='auto',
    zmin=-1, zmax=1
)
fig.update_layout(template='plotly_white')
fig.show()


,land_score,dem_score,dni_score,temp_score,pvout_score,dso_score,pv_score
land_score,1.000,-0.210,0.285,0.178,0.151,0.003,0.022
dem_score,-0.210,1.000,-0.438,-0.844,-0.168,0.031,0.127
dni_score,0.285,-0.438,1.000,0.462,0.355,0.002,0.036
temp_score,0.178,-0.844,0.462,1.000,0.614,-0.011,-0.073
pvout_score,0.151,-0.168,0.355,0.614,1.000,0.078,0.096
dso_score,0.003,0.031,0.002,-0.011,0.078,1.000,0.350
pv_score,0.022,0.127,0.036,-0.073,0.096,0.350,1.000


from matplotlib import pyplot as plt
_df_0['land_score'].plot(kind='hist', bins=20, title='land_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['dem_score'].plot(kind='hist', bins=20, title='dem_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['dni_score'].plot(kind='hist', bins=20, title='dni_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['temp_score'].plot(kind='hist', bins=20, title='temp_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='temp_score', y='pvout_score', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['land_score'].plot(kind='line', figsize=(8, 4), title='land_score')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['dem_score'].plot(kind='line', figsize=(8, 4), title='dem_score')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['dni_score'].plot(kind='line', figsize=(8, 4), title='dni_score')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['temp_score'].plot(kind='line', figsize=(8, 4), title='temp_score')
plt.gca().spines[['top', 'right']].set_visible(False)

ERROR: Ignored the following yanked versions: 1.0.1
ERROR: Could not find a version that satisfies the requirement pymcdm==1.6.8 (from versions: 1.0.0, 1.0.1a0, 1.0.2, 1.0.3, 1.0.3.post1, 1.0.3.post2, 1.0.4, 1.0.5, 1.1.0, 1.2.0, 1.2.1, 1.3.0, 1.3.1)
ERROR: No matching distribution found for pymcdm==1.6.8


AttributeError: module 'pymcdm' has no attribute '__version__'